<a href="https://colab.research.google.com/github/AymanMansur/Insider-threat-detection-using-cert-dataset-Logon-/blob/main/session_duration_non_working_hour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [ ]:
import pandas as pd
from datetime import datetime, time

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/dataset/single user dataset/DNS1758.xlsx', sheet_name='Sheet1')

***Convert the "date" column in df to actual datetime objects so we can perform time-based operations***

In [ ]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

***Sort the DataFrame by 'date' in ascending order and reset the index to ensure a clean 0-based index***

In [ ]:
# Sort by date
df = df.sort_values('date').reset_index(drop=True)

# **Tracking Logons and Logoffs to Build Sessions**

*   **pending_logons:** A dictionary used to keep track of logon events that haven't yet been paired with a logoff
*   **sessions:** A list that will store each completed session (a pair of logon and logoff events)
*   The code loops through each row in the DataFrame

---

*   **For logon event:** A key is created based on the user and the PC ((row['user'],row['pc'])). The logon time (from the 'date' column) is stored in the pending_logons dictionary using this key.

---
*   **For logoff event:** The same key is generated. If the key exists in pending_logons (i.e there was a corresponding logon), the logon time is retrieved and removed from the dictionary.
*   A session dictionary is created with the user, pc, start (logon time), and the end (logoff time) and is appended to the session list.








In [ ]:
# Track pending logons and process sessions
pending_logons = {}
sessions = []

for idx, row in df.iterrows():
    if row['activity'] == 'Logon':
        key = (row['user'], row['pc'])
        pending_logons[key] = row['date']
    elif row['activity'] == 'Logoff':
        key = (row['user'], row['pc'])
        if key in pending_logons:
            logon_time = pending_logons.pop(key)
            logoff_time = row['date']
            sessions.append({
                'user': row['user'],
                'pc': row['pc'],
                'start': logon_time,
                'end': logoff_time
            })

# **Function to calculate non working hours overlap**

***Purpose: This function calculates the total amount of time between start_dt and end_dt tht falls into "non-working hours"***

---
**Non-working hour**

1.   from 12:00:00 AM to 05:59:50 AM
2.   from 18:00:01 PM to 11:59:59 PM

---
Logic:

*   The fucntion initializes a zero Timedelta for duration
*   It then loops day by day from the start date to the end date
*   For each day it defines the two non-working periods
*   Then calculates the segment of the session that occurs within the current day.
*   Computes the overlapping time between this segment and each non-working period.
*   Adds any overlaps to the total duration
*   Finally the function returns the total non-working duration.

In [ ]:
# Function to calculate non-working overlap
def calculate_non_working_duration(start_dt, end_dt):
    duration = pd.Timedelta(0)
    current_day = start_dt.date()
    end_day = end_dt.date()

    while current_day <= end_day:
        day_start = datetime.combine(current_day, time(0, 0, 0))
        non_work_start1 = datetime.combine(current_day, time(0, 0, 0))
        non_work_end1 = datetime.combine(current_day, time(5, 59, 59))
        non_work_start2 = datetime.combine(current_day, time(18, 0, 1))
        non_work_end2 = datetime.combine(current_day, time(23, 59, 59))

        seg_start = max(start_dt, day_start)
        seg_end = min(end_dt, datetime.combine(current_day, time(23, 59, 59)))

        if seg_start >= seg_end:
            current_day += pd.Timedelta(days=1)
            continue

        # Overlap with first period (00:00-05:59:59)
        overlap_start = max(seg_start, non_work_start1)
        overlap_end = min(seg_end, non_work_end1)
        if overlap_start < overlap_end:
            duration += overlap_end - overlap_start

        # Overlap with second period (18:00:01-23:59:59)
        overlap_start = max(seg_start, non_work_start2)
        overlap_end = min(seg_end, non_work_end2)
        if overlap_start < overlap_end:
            duration += overlap_end - overlap_start

        current_day += pd.Timedelta(days=1)

    return duration

# **Calculate and Aggregate Non-working durations per day**
*   Iterating over each session: For every session (which includes the start and end times)
1.   Determine the overall duration: although the variable duration is computed, the code then breaks the session into individual days.
2.   Breaking down by day: it iterates from the start day to the end day of session and for each day it defines the start (day_start) and end (day_end) times of the day.
3.   It computes the portion of the session that falls within that day (seg_start and seg_end)
4. It calculates how much of that day falls within non-working hours using the same calculate_non_working_duration function.
5. it formats the day as a string (YYYY-MM-DD) and accumulates the non-working time in the daily_totals dictionary








In [ ]:
# Calculate non-working durations per session and aggregate by date
daily_totals = {}

for session in sessions:
    start = session['start']
    end = session['end']
    duration = calculate_non_working_duration(start, end)

    current_day = start.date()
    end_day = end.date()
    while current_day <= end_day:
        day_start = datetime.combine(current_day, time(0, 0, 0))
        day_end = datetime.combine(current_day, time(23, 59, 59))
        seg_start = max(start, day_start)
        seg_end = min(end, day_end)

        day_duration = calculate_non_working_duration(seg_start, seg_end)
        day_str = current_day.strftime('%Y-%m-%d')
        if day_str not in daily_totals:
            daily_totals[day_str] = pd.Timedelta(0)
        daily_totals[day_str] += day_duration

        current_day += pd.Timedelta(days=1)

# **Creating a result Dataframe and saving to excel**


In [ ]:
# Convert to DataFrame and save to Excel
result_df = pd.DataFrame(
    sorted(daily_totals.items()),
    columns=['Date', 'Duration']
)


In [ ]:
# Convert Timedelta to HH:MM:SS format
result_df['Duration'] = result_df['Duration'].apply(
    lambda x: str(x).split('.')[0] if pd.notnull(x) else '00:00:00'
)

In [ ]:
# Save to Excel
result_df.to_excel('/content/drive/MyDrive/dataset/single user dataset/session_duration_off_workinghour.xlsx', index=False)

print("Report saved to non_working_hours_report.xlsx")

Report saved to non_working_hours_report.xlsx
